In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0
n = 500

data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [5]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
res = inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=0.9821224699691389, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=1.982122469969139, v=0.0019999996000000823))
 Marginal(NormalMeanVariance{Float64}(μ=2.982122469969138, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=3.9821224699691378, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=4.982122469969138, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=5.982122469969138, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=6.982122469969139, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=7.982122469969138, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=8.982122469969136, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=9.982122469969138, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=10.982122469969134, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64

In [6]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  14.57 MiB
  allocs estimate:  222100
  --------------
  minimum time:     17.049 ms (0.00% GC)
  median time:      17.843 ms (0.00% GC)
  mean time:        19.730 ms (9.19% GC)
  maximum time:     25.998 ms (21.25% GC)
  --------------
  samples:          254
  evals/sample:     1